# FAILING Currently

In [ ]:
import azureml.core
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
# Load secrets from json file
import json
settings= {}
with open('./settings.json') as f:
    settings = json.load(f)

# Name of SQL server
server_name = settings["SQL_SERVERNAME"]
 # Name of SQL database
database_name = settings["SQL_DATABASENAME"]
# SQL Credential. You can also use service principal.
# See https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore(class)?view=azure-ml-py#register-azure-sql-database-workspace--datastore-name--server-name--database-name--tenant-id-none--client-id-none--client-secret-none--resource-url-none--authority-url-none--endpoint-none--overwrite-false--username-none--password-none----kwargs-
sql_user = settings["SQL_USER"]
sql_password = settings["SQL_PASS"]

## Select or create an SQL Store

In [ ]:
sql_datastore_name="store_in_sql_datastore"

from azureml.core.datastore import Datastore
from msrest.exceptions import HttpOperationError
try:
    sql_datastore = Datastore.get(ws, sql_datastore_name)
    print("Found sql database datastore with name: %s" % sql_datastore_name)
except HttpOperationError:
    sql_datastore = Datastore.register_azure_sql_database(
        workspace=ws,
        datastore_name=sql_datastore_name,
        server_name=server_name,
        database_name=database_name,
        username=sql_user,
        password=sql_password)
    print("Registered sql databse datastore with name: %s" % sql_datastore_name)


In [ ]:
# Define an insert
from azureml.data.sql_data_reference import SqlDataReference

sql_query_data_ref = SqlDataReference(
    datastore=sql_datastore,
    data_reference_name="sql_insert_data",
    sql_query="insert into [dbo].[tblExample](id, name) values (1,\"this is a test\")")

In [ ]:
# Get fake output
from azureml.data.data_reference import DataReference
blob_storage_name = "test_datastore"
blob_data_ref = DataReference(
    datastore=Datastore.get(ws,blob_storage_name),
    data_reference_name="blob_test_data",
    path_on_datastore="testdata")

blob_data_ref = SqlDataReference(
    datastore=sql_datastore,
    data_reference_name="sql_select_data",
    sql_query="Select * from [dbo].[tblExample]")


# Get compute
from azureml.exceptions import ComputeTargetException
from azureml.core.compute import DataFactoryCompute
data_factory_name = 'adftest'

def get_or_create_data_factory(workspace, factory_name):
    try:
        return DataFactoryCompute(workspace, factory_name)
    except ComputeTargetException as e:
        if 'ComputeTargetNotFound' in e.message:
            print('Data factory not found, creating...')
            provisioning_config = DataFactoryCompute.provisioning_configuration()
            data_factory = ComputeTarget.create(workspace, factory_name, provisioning_config)
            data_factory.wait_for_completion()
            return data_factory
        else:
            raise e
            
data_factory_compute = get_or_create_data_factory(ws, data_factory_name)

print("Setup Azure Data Factory account complete")

In [ ]:
from azureml.pipeline.steps import DataTransferStep
fake_data_transfer = DataTransferStep(
    name="fake_transfer_to_execute_sql",
    source_data_reference=sql_query_data_ref,
    destination_data_reference=blob_data_ref,
    compute_target=data_factory_compute.name)

In [ ]:
# THIS FAILS
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment
pipeline_01 = Pipeline(
    description="fake_transfer_to_execute_sql",
    workspace=ws,
    steps=[fake_data_transfer])

pipeline_run_01 = Experiment(ws, "fake_data_transfer").submit(pipeline_01)
pipeline_run_01.wait_for_completion()